In [1]:
from common.common_imports import *
from sqlalchemy import create_engine, Table, Index, MetaData, select, distinct, exists
from sqlalchemy.sql import and_, desc
from data.time_encoder import TIME_ENCODER
from data.h5py import H5PY_CONTROLLER
from data.history import HISTORY_DATA
from data.info import INFO_DATA


In [2]:
class DB:
    def __init__(self) -> None:
        self.engine = create_engine('sqlite:////home/kanesweet/kanesweet/stonks.0.1/stonks.db')
        self.connection = self.engine.connect()
        self.metadata = MetaData(bind=self.engine)
        self.history_table = Table('history', self.metadata, autoload=True, autoload_with=self.engine)
        self.attached = False

db = DB()

In [3]:
h5py = H5PY_CONTROLLER()
tickers_idx = pd.read_csv("data/csv/tickers_idx.csv") #7243
to_remove = []
for i, row in tickers_idx.iterrows():
    symbol = row["ticker"]
    
    query = (select(
                db.history_table.c.Date,
                db.history_table.c.Open,
                db.history_table.c.High,
                db.history_table.c.Low,
                db.history_table.c.Close,
                db.history_table.c.Volume,
            )
            .where(db.history_table.c.Ticker == symbol)
            .order_by(db.history_table.c.Date)
            )

    df = pd.read_sql(query, db.connection)
        
    df['year'] = df["Date"].dt.year
    df['month'] = df["Date"].dt.month
    df['day'] = df["Date"].dt.day
    df = df.drop(columns=["Date"])

    array = df.to_numpy()

    # add broadcasted company info embeddings
    # info = INFO_DATA().info_array_from_index(HISTORY_DATA().ticker_stoi(symbol))
    # info = info.reshape(1, -1)
    # info = np.broadcast_to(info, (array.shape[0], info.shape[1]))
    # array = np.concatenate((array, info), axis=1)

    h5py.set_dataset(symbol+"_history", array)

[56.  6. 51.]
[58.  6.  6.]
[ 9.  1. 49.]
[56.  5. 20.]
[ 56.   8. 125.]
[56.  1.  5.]
[56.  4. 74.]
[ 56.   9. 134.]
[56.  6. 69.]
[56.  1.  9.]
[56.  9. 34.]
[ 56.   8. 125.]
[  6.  12. 151.]
[56.  4. 73.]
[56.  5. 20.]
[ 56.   5. 105.]
[56.  4. 75.]
[56.  5. 20.]
[58.  2. 14.]
[ 56.   1. 129.]
[56.  5. 18.]
[56.  4. 44.]
[ 56.   8. 125.]
[ 6.  9. 35.]
[56.  5. 82.]
[ 6.  5. 20.]
[56.  5. 20.]
[56.  4. 72.]
[58.  6. 84.]
[56.  5. 20.]
[  6.   5. 111.]
[ 56.   1. 120.]
[ 56.   1. 135.]
[56.  5. 15.]
[56.  5. 20.]
[  4.   4. 119.]
[  4.   4. 119.]
[56.  5. 15.]
[56.  5. 81.]
[56.  5. 18.]
[50.  5. 20.]
[56.  9. 48.]
[56.  9. 69.]
[56.  1. 86.]
[56.  9. 69.]
[24.  9. 48.]
[56.  4. 75.]
[ 56.   6. 140.]
[56.  5. 15.]
[56.  4. 73.]
[ 56.   8. 125.]
[ 56.   8. 125.]
[56.  5. 20.]
[56.  5. 20.]
[  6.   5. 111.]
[56.  7. 93.]
[56.  1. 69.]
[56.  4. 58.]
[55.  5. 15.]
[56.  9. 35.]
[ 56.   8. 125.]
[56.  1. 54.]
[56.  9. 35.]
[56.  6. 76.]
[ 56.   9. 134.]
[56.  5. 20.]
[ 56.   2. 108.]
[56. 

KeyboardInterrupt: 

In [ ]:
symbol = "AAPL"
array = h5py.get_dataset(symbol+"_history")
df = pd.DataFrame(array, columns=[
    'Open', 
    'High', 
    'Low', 
    'Close',
    'Volume', 
    'year',
    'month',
    'day'])
print(df)

            Open        High         Low       Close       Volume    year  \
0       0.795873    0.853737    0.771683    0.849468  535796800.0  2000.0   
1       0.821485    0.839508    0.767889    0.777850  512377600.0  2000.0   
2       0.787335    0.839034    0.781644    0.789232  778321600.0  2000.0   
3       0.805359    0.811999    0.720933    0.720933  767972800.0  2000.0   
4       0.732317    0.766466    0.724728    0.755083  460734400.0  2000.0   
...          ...         ...         ...         ...          ...     ...   
5897  181.270004  183.889999  180.970001  183.789993   54274900.0  2023.0   
5898  182.800003  184.149994  182.440002  183.309998   54929100.0  2023.0   
5899  183.369995  184.389999  182.020004  183.949997   57462900.0  2023.0   
5900  183.960007  186.520004  183.779999  186.009995   65433200.0  2023.0   
5901  186.729996  186.990005  184.270004  184.919998  101235600.0  2023.0   

      month   day  
0       1.0   3.0  
1       1.0   4.0  
2       1.0   5